In [120]:
import numpy as np
import pandas as pd
import math
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import rc
import seaborn as sns
from scipy import stats
from scipy.optimize import curve_fit
from scipy.integrate import odeint
from scipy.integrate import solve_ivp
import time

# Directions for use:
    
1. Specify benthic conditions **(AltSS, MacroDom, CoralDom)** in code block 3
2. Use **AltSS_finder function** to add one, two, or three fish to system

## ODE System

In [121]:
def dNdt(t, N, rC, rM, rT, αCT, αMT, αMC, μC, μM, μT, #benthos
         hB, nB, sB, σB, θB, eB, pB,                  #browsers
         hG, nG, sG, σG, θG, eG, pG,                  #grazers
         hS, nS, sS, σS, θS, eS, pS):                 #scrapers
    
    C,M,T,B,G,S = N
    
    dC = (rC*C*(1-C-M-T) + αCT*rC*C*T - αMC*rM*C*M - μC*C)*dt
    dM = (rM*M*(1-C-M-T) + αMT*rM*T*M + αMC*rM*C*M - μM*M - ((hB*B*M)/((hB*nB*M)+1)))*dt
    dT = (rT*T*(1-C-M-T) - αCT*rC*C*T - αMT*rM*T*M - μT*T - ((hG*G*T)/((hG*nG*T)+1)) - ((hS*S*T)/((hS*nS*T)+1)))*dt
    
    dB = (sB*B*(1-(B/((1-σB)+(σB*C)))) + θB*B - eB*B - pB*B)*dt
    dG = (sG*G*(1-(G/((1-σG)+(σG*C)))) + θG*G - eG*G - pG*G)*dt
    dS = (sS*S*(1-(S/((1-σS)+(σS*C)))) + θS*G - eS*G - pS*G)*dt

    return dC, dM, dT, dB, dG, dS

## Initializing Benthos and Timesteps

1. Comment out conditions **NOT** being tested

In [122]:
μM_MacroDom_11 = 0.11
μM_AltSS_12 = 0.12
μM_CoralDom_13 = 0.13

#Macroalgae Dominant------------------------------
#rC, rM, rT, αCT, αMT, αMC, μC, μM, μT = 0.2, 0.35, 5, 0.25, 0.9, 0.1, 0.05, μM_MacroDom_11, 10

#Alternative Stable States------------------------
rC, rM, rT, αCT, αMT, αMC, μC, μM, μT = 0.2, 0.35, 5, 0.25, 0.9, 0.1, 0.05, μM_AltSS_12, 10

#Coral Dominant-----------------------------------
#rC, rM, rT, αCT, αMT, αMC, μC, μM, μT = 0.2, 0.35, 5, 0.25, 0.9, 0.1, 0.05, μM_CoralDom_13, 10


In [123]:
dt = 1
NUMSTEPS = 5000
NUMYEARS = int(NUMSTEPS/dt)
time_points = np.linspace(0, NUMYEARS, NUMSTEPS+1)

h_val = np.linspace(0,1,11)
B0, G0, S0 = 0.1, 0.1, 0.1

## Function to find AltSS at increasing rates of h (herbivory rate)

In [124]:
def AltSS_finder(scenario): 
    
    outputs = []
    outputs_prime = []
    
    for h in h_val:
        print ('h='+str(h))
        
        for C0 in np.linspace(0.01,1,11):
            print ('C0='+str(C0))

            for M0 in np.linspace(0.01,1,11):

                for T0 in np.linspace(0.01,1,11):

                    if (C0 + M0 + T0 <= 1):
    
                        #Baseline Benthos (No Herbivores)------------------------------
                        if scenario == 'baselinebenthos':
                            hB, nB, sB, σB, θB, eB, pB = 0, 0, 0, 0, 0, 0, 0
                            hG, nG, sG, σG, θG, eG, pG = 0, 0, 0, 0, 0, 0, 0
                            hS, nS, sS, σS, θS, eS, pS = 0, 0, 0, 0, 0, 0, 0

                        #Browsers-----------------------------------------------------
                        elif scenario == 'browser':
                            hB, nB, sB, σB, θB, eB, pB = h, 1, 0.1, 1, 0, 0, 0.05
                            hG, nG, sG, σG, θG, eG, pG = 0, 0, 0, 0, 0, 0, 0
                            hS, nS, sS, σS, θS, eS, pS = 0, 0, 0, 0, 0, 0, 0

                        #Grazers-----------------------------------------------------
                        elif scenario == 'grazer':
                            hG, nG, sG, σG, θG, eG, pG = h, 1, 0.2, 1, 0, 0, 0.05
                            hB, nB, sB, σB, θB, eB, pB = 0, 0, 0, 0, 0, 0, 0
                            hS, nS, sS, σS, θS, eS, pS = 0, 0, 0, 0, 0, 0, 0

                        #Scrapers-----------------------------------------------------
                        elif scenario == 'scraper':
                            hS, nS, sS, σS, θS, eS, pS = h, 1, 0.5, 1, 0, 0, 0.05
                            hB, nB, sB, σB, θB, eB, pB = 0, 0, 0, 0, 0, 0, 0
                            hG, nG, sG, σG, θG, eG, pG = 0, 0, 0, 0, 0, 0, 0

                        #Browsers & Grazers-------------------------------------------
                        elif scenario == 'browser_grazer':
                            hB, nB, sB, σB, θB, eB, pB = h, 1, 0.1, 1, 0, 0, 0.05
                            hG, nG, sG, σG, θG, eG, pG = h, 1, 0.2, 1, 0, 0, 0.05
                            hS, nS, sS, σS, θS, eS, pS = 0, 0, 0, 0, 0, 0, 0

                        #Browsers & Scrapers------------------------------------------
                        elif scenario == 'browser_scraper':
                            hB, nB, sB, σB, θB, eB, pB = h, 1, 0.1, 1, 0, 0, 0.05
                            hS, nS, sS, σS, θS, eS, pS = h, 1, 0.5, 1, 0, 0, 0.05
                            hG, nG, sG, σG, θG, eG, pG = 0, 0, 0, 0, 0, 0, 0

                        #Grazers & Scrapers------------------------------------------
                        elif scenario == 'grazer_scraper':
                            hG, nG, sG, σG, θG, eG, pG = h, 1, 0.2, 1, 0, 0, 0.05
                            hS, nS, sS, σS, θS, eS, pS = h, 1, 0.5, 1, 0, 0, 0.05
                            hB, nB, sB, σB, θB, eB, pB = 0, 0, 0, 0, 0, 0, 0

                        #Browsers, Grazers, & Scrapers--------------------------------
                        elif scenario == 'all_herbivores':
                            hB, nB, sB, σB, θB, eB, pB = h, 1, 0.1, 1, 0, 0, 0.05
                            hG, nG, sG, σG, θG, eG, pG = h, 1, 0.2, 1, 0, 0, 0.05
                            hS, nS, sS, σS, θS, eS, pS = h, 1, 0.5, 1, 0, 0, 0.05
       

                        sol = solve_ivp(dNdt, [0,NUMYEARS], [C0, M0, T0, B0, G0, S0], 
                                        method = 'RK45', args = 
                                        (rC, rM, rT, αCT, αMT, αMC, μC, μM, μT,                              #benthos
                                        hB, nB, sB, σB, θB, eB, pB,                                          #browsers
                                        hG, nG, sG, σG, θG, eG, pG,                                          #grazers
                                        hS, nS, sS, σS, θS, eS, pS),                                         #scrapers
                                        dense_output=True)

                        N = sol.sol(time_points)

                        C_array = N[0,:]
                        M_array = N[1,:]
                        T_array = N[2,:]
                        B_array = N[3,:]
                        G_array = N[4,:]
                        S_array = N[5,:]

                        outputs.append((rC, rM, rT, αCT, αMT, αMC, μC, μM, μT,                              #benthos
                                        hB, nB, sB, σB, θB, eB, pB,                                         #browsers
                                        hG, nG, sG, σG, θG, eG, pG,                                         #grazers
                                        hS, nS, sS, σS, θS, eS, pS,                                         #scrapers
                                        C_array, M_array, T_array, B_array, G_array, S_array))
                            
                        outputs_prime.append((rC, rM, rT, αCT, αMT, αMC, μC, μM, μT,                        #benthos
                                        hB, nB, sB, σB, θB, eB, pB,                                         #browsers
                                        hG, nG, sG, σG, θG, eG, pG,                                         #grazers
                                        hS, nS, sS, σS, θS, eS, pS,                                         #scrapers
                                        C_array[-1], M_array[-1], T_array[-1], B_array[-1], G_array[-1], S_array[-1]))

    df_timeseries = pd.DataFrame(outputs, columns=['rC', 'rM', 'rT', 'αCT', 'αMT', 'αMC', 'μC', 'μM', 'μT', #benthos
                                        'hB', 'nB', 'sB', 'σB', 'θB', 'eB', 'pB',                           #browsers
                                        'hG', 'nG', 'sG', 'σG', 'θG', 'eG', 'pG',                           #grazers
                                        'hS', 'nS', 'sS', 'σS', 'θS', 'eS', 'pS',                           #scrapers
                                        'C_array', 'M_array', 'T_array', 'B_array', 'G_array', 'S_array'])
    
    
    df_prime = pd.DataFrame(outputs_prime, columns=['rC', 'rM', 'rT', 'αCT', 'αMT', 'αMC', 'μC', 'μM', 'μT', #benthos
                                        'hB', 'nB', 'sB', 'σB', 'θB', 'eB', 'pB',                            #browsers
                                        'hG', 'nG', 'sG', 'σG', 'θG', 'eG', 'pG',                            #grazers
                                        'hS', 'nS', 'sS', 'σS', 'θS', 'eS', 'pS',                            #scrapers
                                        'C_prime', 'M_prime', 'T_prime', 'B_prime', 'G_prime', 'S_prime'])
    
    return(df_timeseries, df_prime)

# Running Function
1. Specify which **scenario (i.e. which herbivores are included)** and **save as pickle file**

## Browser
#### AltSS (μM=12)

In [ ]:
start_time = time.perf_counter()

Browsers_AltSS_μM12_hB0to1_030125  = AltSS_finder(scenario = 'browser')

Browsers_AltSS_μM12_hB0to1_030125[0].to_pickle('/Users/gails/Desktop/Team Sediment/Chapter 1 - Theory/Current Analyses/timeseries_Browsers_AltSS_μM12_hB0to1_030125')
Browsers_AltSS_μM12_hB0to1_030125[1].to_pickle('/Users/gails/Desktop/Team Sediment/Chapter 1 - Theory/Current Analyses/equilibria_Browsers_AltSS_μM12_hB0to1_030125')

end_time = time.perf_counter()
(end_time - start_time)/60

h=0.0
C0=0.01
C0=0.109
C0=0.20800000000000002


#### MacroDom (μM=11)

In [ ]:
start_time = time.perf_counter()

Browsers_MacroDom_μM11_hB0to1_030125  = AltSS_finder(scenario = 'browser')

Browsers_MacroDom_μM11_hB0to1_030125[0].to_pickle('/Users/gails/Desktop/Team Sediment/Chapter 1 - Theory/Current Analyses/timeseries_Browsers_MacroDom_μM11_hB0to1_030125')
Browsers_MacroDom_μM11_hB0to1_030125[1].to_pickle('/Users/gails/Desktop/Team Sediment/Chapter 1 - Theory/Current Analyses/equilibria_Browsers_MacroDom_μM11_hB0to1_030125')

end_time = time.perf_counter()
(end_time - start_time)/60

#### CoralDom (μM=13)

In [ ]:
start_time = time.perf_counter()

Browsers_CoralDom_μM13_hB0to1_030125  = AltSS_finder(scenario = 'browser')

Browsers_CoralDom_μM13_hB0to1_030125[0].to_pickle('/Users/gails/Desktop/Team Sediment/Chapter 1 - Theory/Current Analyses/timeseries_Browsers_CoralDom_μM13_hB0to1_030125')
Browsers_CoralDom_μM13_hB0to1_030125[1].to_pickle('/Users/gails/Desktop/Team Sediment/Chapter 1 - Theory/Current Analyses/equilibria_Browsers_CoralDom_μM13_hB0to1_030125')

end_time = time.perf_counter()
(end_time - start_time)/60